In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import sys

import cv2
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from PIL import Image
import plotly.graph_objects as go
import pyransac3d as pyrsc
from tqdm import tqdm
from scipy.spatial import KDTree
import torch
import visu3d as v3d

sys.path.append(os.path.join(os.getcwd(), "dust3r"))
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from lang_sam import LangSAM

from barrelnet.pointnet.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet.pointnet.barrelnet import BarrelNet
from barrelnet.pointnet.data import pts2inference_format
from barrelnet.dust3r_utils import save_dust3r_outs, read_dust3r, resize_to_dust3r
from barrelnet.langsam_utils import display_image_with_masks, display_image_with_boxes
from barrelnet.utils import segment_pc_from_mask, get_local_plane_mask, rotate_pts_to_ax, get_bbox_mask
from barrelnet.synthbarrel import get_cyl_endpoints, get_cylinder_surf

In [ ]:
image_dir = Path("data/dive8-barrel-10-45")

# H, W = (875, 1920)
H, W = (224, 512)
reconstr_dir = Path(f"results/{image_dir.name}-reconstr")
mask_dir = reconstr_dir / "masks"
resizeimg_dir = reconstr_dir / "resized"
imgpaths = sorted(image_dir.glob("*.jpg"))
dust3rout_path = reconstr_dir / "dust3r_out.pth"

In [ ]:
image_dir.parent

In [ ]:
with open(reconstr_dir / "bboxes.pickle", "rb") as f:
    bboxes = pickle.load(f)

In [ ]:
(image_dir.parent / "maskedbbox").mkdir(exist_ok=True, parents=True)
boxed = []
for i, imgpath in enumerate(imgpaths):
    img = Image.open(resizeimg_dir / imgpath.name).convert("RGB")
    imgname = Path(imgpath.name).stem
    bbox = bboxes[i]
    imgnp = np.array(img)
    H, W, _ = imgnp.shape
    masknp = get_bbox_mask(bbox, W, H)
    imgboxed = imgnp * (masknp[..., None] > 0)
    boxed.append(imgboxed)
    Image.fromarray(imgboxed).save((image_dir.parent / "maskedbbox") / f"{imgname}-boxed.jpg")